# 导入库

In [1]:
# encoding: UTF-8
import re
from Bio.Blast.Applications import NcbipsiblastCommandline
from Bio import SeqIO
from Bio import SwissProt
import os
from numpy import argmax
from numpy import *
import re
import scipy.io as sio
import numpy as np


# 1、将fasta格式文件整合为mat文件

In [ ]:

def fasta2mat(fasta_path,mat_name):
    dataset1={}
    path=fasta_path
    for seq_record in SeqIO.parse(path, 'fasta'):
        #print('seq_record:  ',seq_record)
        #print(seq_record.seq)
        #print('{} is calculating pssm'.format(seq_record.id))
    #     #print('')
    #     if os.path.exists(inputfile):
    #         os.remove( inputfile)
    #     ont_hotfile = "".join( ('One_Hot', '_', seq_record.id, '.txt'))
    #     print(ont_hotfile)

    #     SeqIO.write( seq_record,inputfile, 'fasta')

        data = seq_record.seq

        encoded = list()

        encoded = np.array(data)
        #print(onehot_encoded)

        dataset1[seq_record.id]= encoded
        
        savefile=mat_name
        sio.savemat(savefile,dataset1)


    print('PDNA-224_.mat is OK!')

fasta2mat('..\\PDNA-224.fasta','PDNA-224_.mat')


In [2]:
data1 = sio.loadmat('PDNA-224_.mat')
# X = data1['data']
# Y = data1['target']
print(data1)

{'__header__': b'MATLAB 5.0 MAT-file Platform: nt, Created on: Tue Oct 16 22:24:09 2018', '__version__': '1.0', '__globals__': [], '1A02_N': array(['M', 'R', 'G', 'S', 'H', 'H', 'H', 'H', 'H', 'H', 'T', 'D', 'P',
       'H', 'A', 'S', 'S', 'V', 'P', 'L', 'E', 'W', 'P', 'L', 'S', 'S',
       'Q', 'S', 'G', 'S', 'Y', 'E', 'L', 'R', 'I', 'E', 'V', 'Q', 'P',
       'K', 'P', 'H', 'H', 'R', 'A', 'H', 'Y', 'E', 'T', 'E', 'G', 'S',
       'R', 'G', 'A', 'V', 'K', 'A', 'P', 'T', 'G', 'G', 'H', 'P', 'V',
       'V', 'Q', 'L', 'H', 'G', 'Y', 'M', 'E', 'N', 'K', 'P', 'L', 'G',
       'L', 'Q', 'I', 'F', 'I', 'G', 'T', 'A', 'D', 'E', 'R', 'I', 'L',
       'K', 'P', 'H', 'A', 'F', 'Y', 'Q', 'V', 'H', 'R', 'I', 'T', 'G',
       'K', 'T', 'V', 'T', 'T', 'T', 'S', 'Y', 'E', 'K', 'I', 'V', 'G',
       'N', 'T', 'K', 'V', 'L', 'E', 'I', 'P', 'L', 'E', 'P', 'K', 'N',
       'N', 'M', 'R', 'A', 'T', 'I', 'D', 'C', 'A', 'G', 'I', 'L', 'K',
       'L', 'R', 'N', 'A', 'D', 'I', 'E', 'L', 'R', 'K', 'G', 'E', 

# 2、滑窗处理序列并保存

In [7]:


# param ws: slip windown size
def establishSegDataset(ws, savefile):
    X = []
    Y = []
    k = 0
    t = 0
    
    datafile = 'PDNA-224_.mat'
    matfile = sio.loadmat(datafile)

    # read fasta file
    #fastafile = 'PDNA-224.fasta'
    #seq_records = SeqIO.parse(fastafile)

    # build slip window with size 11*2+1=23
    
    # read 'PDNA-224-binding-sites.txt'
    with open('..\\data\\PDNA-224-binding-sites.txt', 'r') as pbsreader:
        for line in pbsreader:
            line = line.strip()
            
            
           
            if '>' in line:         
                sid = line[1:]   #sid  蛋白质序列ID
                #print(sid)
                
                p = matfile[sid]   #读取蛋白质序列ID所对应的序列
                #print(p.shape)
                seqlen = len(p)
                #print(seqlen)
                line_num=ws*2+1 #行数
                
                for j in range(seqlen):
                    #create a array000000000000000000000000000000000000
                    d = np.ndarray(shape=(line_num,),dtype='<U1')

                    
                    if j < ws:
                        d[0:ws-j] = p[j-ws:]
                        d[ws-j:2*ws+1] = p[0: ws+j+1]
                    elif j > seqlen - ws -1:
                        d[0:ws] = p[j-ws:j]
                        d[ws:ws + seqlen -j] = p[j:]
                        d[ws+seqlen-j:] = p[0:ws-seqlen+j+1]
                    else:
                        d[::]=p[j-ws:j+ws+1]
                        
#                     for dl in range(line_num):
#                         for dr in range(20):
#                             if d[dl,dr]<0:
                                
#                                 d[dl,dr]=d[dl,dr]+256
                                
#                     print('')
#                     print(d)
#                     print('')
                    X.insert(k,d)
                    Y.append([-1])
                    
                    k += 1
                    #print('k={},t={}'.format(k,t))
            else:
                sites = line.split() #结合位点位置
                #print(sites)
                for s in sites:
                    idx = eval(s)
                    #print(idx)
                    #print('t={},idx={}'.format(t,idx))
                    #print(t+idx-1)
                    Y[t + idx-1] = [1]
                    #print(t + idx-1 )
                t=k        
    #save benchmark data set
    
    dataset={}
    dataset['data']=X
    dataset['target'] = Y
    sio.savemat(savefile,dataset)
    
establishSegDataset(11,'PDNA-224-seg-11.mat') 
print('PDNA-224-seg-11.mat is OK!')

PDNA-224-seg-11.mat is OK!


# 3、对滑窗后的序列进行5位二进制编码

In [8]:
def Seg2Bin(segfile, savefile):
    
    data = sio.loadmat(segfile)
    X = data['data']
    Y = data['target']
    
    #定义编码模型
    dict = { 'P':'00001', 'Q': '00100', 'R': '00110', 'Y': '01100', 'W': '01110',
            'T': '10000', 'M': '10011', 'N': '10101', 'V': '11010', 'E': '11101',
            'L': '00011', 'H': '00101', 'S': '01001', 'F': '01011', 'C': '01111',
            'I': '10010', 'K': '10100', 'A': '11001', 'D': '11100', 'G': '11110', 'X': '00000'} #将乱码X设为00000


    XX=np.ndarray(shape=(57348,1),dtype='<U115')

    for i in range(len(X)):
        bin_=''
        for j in range(23):
            stra=str(dict[X[i,j]]) #将编码转化为字符串
            bin_ = bin_ + stra


        XX[i,0]=bin_

    dataset1={}
    dataset1['data']=XX
    dataset1['target'] = Y
    sio.savemat(savefile,dataset1)


Seg2Bin('PDNA-224-seg-11.mat','PDNA-224-5-Bin-11.mat')

print('PDNA-224-5-Bin-11.mat is OK!')

PDNA-224-5-Bin-11.mat is OK!


# 测试：单条序列元胞自动机演化

In [ ]:
# 
rule_84_nb = ['111','110','101','100','011','010','001','000']  
rule_84_bit = ['0',  '1',  '0',  '1',  '0',  '1',  '0',  '0']  #84代表二进制 010100100
#

ab=bbbb[len(bbbb)-1]+bbbb+bbbb[0]  #演化规则：将最后一位补到头部，第一位补到尾部 则长度为115+2=117
#bbbb[0:115]
#ab[0:117]

print('',bbbb)
print(ab)
new_ab=''
for k in range(len(ab)-2):
    nb = ab[k:k+3]  #滑窗得到邻居
    index_= rule_84_nb.index(nb)
    rule_bit = rule_84_bit[index_]
    new_ab = new_ab + rule_bit
print(nb)
print(index_,rule_bit)
print(new_ab)


# 4.1 元胞自动机演化模型

In [3]:
rule_84_nb = ['111','110','101','100','011','010','001','000']  
rule_84_bit = ['0',  '1',  '0',  '1',  '0',  '1',  '0',  '0']
def Cellular_Automata(stra):

    xs=stra
    ab=xs[len(xs)-1]+xs+xs[0]  #演化规则：将最后一位补到头部，第一位补到尾部 则长度为115+2=117
    
    #xs[0:115]
    #ab[0:117]
    new_ab=''
    
    for k in range(len(ab)-2):
        nb = ab[k:k+3]  #滑窗得到邻居
        index_= rule_84_nb.index(nb)
        rule_bit = rule_84_bit[index_]
        new_ab = new_ab + rule_bit
    
    return new_ab



In [10]:

data1 = sio.loadmat('PDNA-224-5-Bin-11.mat')
X = data1['data']
Y = data1['target']
print(Y)

[[-1]
 [-1]
 [-1]
 ...
 [-1]
 [-1]
 [-1]]


# 4.2 对二进制序列进行元胞自动机处理

In [11]:
def Bin2Cellauto(binfile,epochs,savefile):
    
    data1 = sio.loadmat('PDNA-224-5-Bin-11.mat')
    X = data1['data']
    Y = data1['target']
    
    XSS=[]
    epoch=epochs #定义迭代次数
    t = 0
    for k in range(len(X)):
        bin_5=X[k,0]

        XS=np.zeros(shape=(epoch,len(bin_5)), dtype=np.int16)

        cellauto_bin=bin_5
        
        for j in range(epoch): 

            cellauto_bin = Cellular_Automata(cellauto_bin)
            result = list(cellauto_bin)   
            results = np.array(result, dtype=np.int16) #将list中的字符型转变为数值型
#             if j>399 :
            
            XS[j]=results

            #print(cellauto_bin)

        XSS.insert(t,XS)
        t+=1

    dataset={}
    dataset['data']=XSS
    dataset['target'] = Y
    sio.savemat(savefile,dataset)
    
Bin2Cellauto('PDNA-224-5-Bin-11.mat',100,'PDNA-224-Cellauto-11-100-2.mat')

print('PDNA-224-Cellauto-11-100.mat is OK!')

PDNA-224-Cellauto-11-100.mat is OK!


In [ ]:
data2 = sio.loadmat('PDNA-224-Cellauto-11-200.mat')
X2 = data2['data']
Y2 = data2['target']


In [ ]:

X2[57347,0:3]

# 5、划分为正负样本集

In [ ]:

import numpy as np
import scipy.io as sio
from sklearn.model_selection import KFold
import tensorflow as tf

X_N = []
Y_N = []
X_P = []
Y_P = []

data = sio.loadmat('PDNA-224-Cellauto-11-115.mat')
X = data['data']
data_size=len(X)
Y = data['target']

#print(type(X))



for sss in range(data_size):
    if Y[sss][0] == 1: #判定是否为负样本
        #print(X[sss],Y[sss])
        X_N.append(X[sss])
        Y_N.append(Y[sss])
        #print(X[sss])
    else:
        X_P.append(X[sss])
        Y_P.append(Y[sss])
    
#将正样本保存到mat文件中
P_dataset={}
P_dataset['p_data']=X_P
P_dataset['p_target'] = Y_P
sio.savemat('..\\data\\PDNA-224-Cellauto-11-115-P.mat',P_dataset)

#将负样本保存到mat文件中
N_dataset={}
N_dataset['n_data']=X_N
N_dataset['n_target'] = Y_N
sio.savemat('..\\data\\PDNA-224-Cellauto-11-115-N.mat',N_dataset)
print('yes!')
  